In [6]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../src')
from preprocess import get_train_test

In [7]:
data = get_train_test(args=0)
data.head()

/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,session_id,registor_number,user_id,sex,age,distance_to_the_store,start_at__hour,test,target,month,day,week
0,ebd85fd0faf8ab75838afff4b83451575844d134cf858c...,0,0,0,37.0,1.0,13,False,0,11,1,0
1,7fb5a45de5c44a638871f08a9e20376e009a7621451fbe...,1,1,0,62.0,7.0,14,False,0,11,1,0
2,4c4cb2fcc5bcb0b0ca2ae0e584c19062982a6164e8686d...,2,2,0,32.0,5.0,12,False,0,11,1,0
3,4c4479d2ab18b2c9170e1132dc51a95332d35b167a3c90...,3,3,1,32.0,8.0,16,False,0,11,1,0
4,93e84423ebb1846b29c27c2869e714a9c6a60e52005919...,4,4,0,67.0,2.0,14,False,0,11,1,0


In [10]:
from sklearn.model_selection import KFold
tmp = np.repeat(np.nan, data.shape[0])

target_enc = data[data['test']==False].groupby('sex')['target'].mean()
tmp[data['test']==True] = data.loc[data['test']==True, 'sex'].map(target_enc)

kf = KFold(n_splits=5, shuffle=True)
for trn, val in kf.split(data[data['test']==False]):
    target_enc = data.iloc[trn].groupby('sex')['target'].mean()
    tmp[val] = data.loc[val, 'sex'].map(target_enc)

data['sex'] = tmp
data['sex'].head()

0    0.092328
1    0.090844
2    0.092983
3    0.057047
4    0.090844
Name: sex, dtype: float64

In [ ]:
data.dtypes

session_id                 object
registor_number          category
user_id                  category
sex                       float16
age                       float16
distance_to_the_store     float16
start_at__hour              int64
test                         bool
target                      uint8
month                    category
day                      category
week                     category
dtype: object

In [ ]:
data['lag_target'] = data.groupby('user_id').target.shift(1)
data.loc[data['user_id']==5]

,session_id,registor_number,user_id,sex,age,distance_to_the_store,start_at__hour,test,target,month,day,week,lag_target
91813,d78288aa5434822d496fdfb113982583c0fd67c4fbb827...,5,5,0.000000,42.0,1.0,17,False,0,11,1,0,NaN
55057,80fe28cee34aa4fe4999c1cc734d5b5e4bcabed196f477...,48,5,0.000000,42.0,1.0,16,False,0,11,2,1,0.0
100514,ebe36cdd2a72ddeb6811c129a936edbc5b34060610b768...,1,5,0.000000,42.0,1.0,10,False,0,11,3,2,0.0
70759,a57d0d59d42fb3cd5e00ce7bf4f3548952446c5bf2df1c...,164,5,0.000000,42.0,1.0,16,False,0,11,4,3,0.0
20477,30464a24defd36d4108736b8b1c3d541c13df7d8523ddd...,65,5,0.000000,42.0,1.0,17,False,0,11,9,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27688,410b929f54fde6785475864dcb4da32b12affbf1694c86...,38,5,0.085815,42.0,1.0,17,True,255,1,25,1,255.0
42668,6435724d97719ff99cb55276932e21c06850d2c8a54570...,71,5,0.085815,42.0,1.0,16,True,255,1,26,2,255.0
35386,5308ae23c549367f95f1e73c70692d89734dbc1f713a86...,13,5,0.085815,42.0,1.0,16,True,255,1,28,4,255.0
77419,b535c79fd899e605feb7cef6c5a5a7dab7770e2cb58a17...,19,5,0.085815,42.0,1.0,12,True,255,1,29,5,255.0


In [ ]:
DATA_PATH = '../input/'
import os

data = pd.read_csv(os.path.join(DATA_PATH, "session.csv"))
test_session = pd.read_csv(os.path.join(DATA_PATH, "test_session.csv"))


In [ ]:
data['test'] = data['session_id'].isin(test_session['session_id'])
data[data['test'] == True]

,session_id,registor_number,user_id,sex,age,distance_to_the_store,start_at__date,start_at__hour,test
2,000280d2156d179a597167b269a69e5af556725eeaabae...,1016,8da8267f34ec7ee3f3b800b089b7e27c16fe8cf5a7e36b...,女性,35~39,17,2022-01-25,13,True
9,00055a0719ae8afefe0a785d4099a98da414b72345b2d9...,1051,79f9d3fa4006cb284677e95194543a3ccfe3db55cf5294...,女性,65~69,3,2022-01-12,21,True
17,0009065eec287b1cedc44425b2b53c82aba347bf9966be...,1117,8b2894cef5e1de80f0b13cc33655f9bf5d5442da3d3aac...,女性,65~69,5,2022-01-16,13,True
27,000efa1223901c67e9abc8030b55218240174b6c793289...,1077,c5bd0a9e3958a1218f6d1b9817f83e7bd02ec93151dc14...,男性,45~49,1,2022-01-25,19,True
30,001019dafd085898d4bcd7631468af2b3ccb6120b71bac...,1099,2ed875b8f5456649d34849ede7cff795513d853c14dc0e...,女性,75~79,3,2022-01-11,11,True
...,...,...,...,...,...,...,...,...,...
109135,fff17a787eadeac8c78421e7fce6f3b918091c6c550be7...,1123,6fb02f2a89a26f6c3f1a317523360782e49a0b2cc324b5...,男性,40~44,5,2022-01-14,17,True
109138,fff2a322f01b391d1b73a9ff5bf878fc2925d26a5db9b9...,1096,dfb734472e76a9ccfb121e219c78fb8950096631aa317c...,男性,30~34,1,2022-01-17,20,True
109149,fff7476be6ba92ceac0c7830d96d92266061433992419b...,1116,4214e9d1709d0a01d045b0918208bbb89a87825b3bae5e...,女性,40~44,3,2022-01-12,10,True
109154,fffa55a2a6d0e29505b48dc6e49e07ba004a312e23fe41...,1059,c973f71eb917860aa4d69594a237b15a87442157e64cc8...,女性,50~54,26,2022-01-30,16,True


In [ ]:
data['age'].unique()

array(['60~64', '55~59', '35~39', '30~34', '25~29', '50~54', '70~74',
       '65~69', '45~49', '不明', '15~19', '75~79', '40~44', nan, '20~24',
       '80~84', '85~89', '10~14', '100~', '0~4', '90~94', '5~9', '95~99'],
      dtype=object)